In [12]:
folder = 'mouse-brain-rna-seq-2019/'
filename = folder + 'matrix.csv'
num_rows = 74900 # Five hundred million
batch_size = 5000

In [13]:
import numpy as np
import pandas as pd
import csv
import tensorflow as tf

In [14]:
def cutoff_index(out_num):
    #take fraction of cell data with highest variance in trimmed means
    df = pd.read_csv(folder+'trimmed_means.csv')
    #df = pd.read_csv(folder+'medians.csv')
    dg = df.var(axis=0)
    dg_sort = dg.sort_values(ascending=False)
    cutoff = dg_sort[out_num]
    gene_names = dg[dg>cutoff].index.values
    #ngenes = len(gene_names)
    dh=dg.copy(deep=True)
    dh[:]=np.arange(len(dh))
    gene_index = dh[dg>cutoff].values
    return gene_names,gene_index

In [15]:
gene_names,gene_index = cutoff_index(10000)

In [4]:
cell_types=pd.read_csv(folder+'metadata.csv',usecols=['subclass_order'])

In [6]:
len(cell_types['subclass_order'].unique())

44

In [19]:
cell_types['subclass_order'].unique()

array([ 5., 13.,  3.,  4.,  8.,  2.,  6.,  7., nan,  9., 15., 27., 14.,
       17., 22., 10., 16., 20., 37., 11., 12., 19., 32., 18., 21., 25.,
       34., 30., 33., 23., 26., 24., 31., 36., 35., 29., 28.,  1., 39.,
       38., 42., 40., 41., 43.])

In [20]:
import tensorflow as tf
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(8000, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(44)
])

In [21]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
# Choose and optimizer and compile a model
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
datafile = filename
chunksize = batch_size
total_cells=num_rows
trainsize=int(total_cells*0.8)

#models = []
#for chunk in pd.read_csv(datafile, chunksize=chunksize):
for epoch in range(10):
    i=0
    in_chunker = pd.read_csv(datafile, chunksize=chunksize, nrows=trainsize, usecols=gene_names) #skiprows=1
    out_chunker = pd.read_csv(folder+'metadata.csv', chunksize=chunksize, nrows=trainsize,usecols=['subclass_order'])

    test_in_chunker = pd.read_csv(datafile, chunksize=chunksize, skiprows=[trainsize], usecols=gene_names)
    test_out_chunker = pd.read_csv(folder+'metadata.csv', chunksize=chunksize, skiprows=[trainsize],usecols=['subclass_order'])
    
    for in_chunk, out_chunk in zip(in_chunker, out_chunker):
        in_chunk=in_chunk.values.astype(int)
        label_chunk = out_chunk.fillna(0).astype(int)#['cell_type_alias_id']#out_chunk['cell_type_alias_label']
        print(in_chunk.shape,label_chunk.shape)
        model.fit(in_chunk, label_chunk, epochs = 1, batch_size = 100)
        #model.fit(in_chunk, label_chunk, steps_per_epoch=60, epochs=5)#,
                 #steps_per_epoch=trainsize / batch_size, epochs=10, verbose=1)
            #train_on_batch(in_chunk, label_chunk)
        i=i+1
        print(epoch,i,i*chunksize,(i*chunksize)/trainsize)

(5000, 10000) (5000, 1)
50/50 [==============================] - 26s 512ms/step - loss: 5860.6738 - accuracy: 0.3832
0 1 5000 0.08344459279038718
(5000, 10000) (5000, 1)
50/50 [==============================] - 26s 526ms/step - loss: 765.4344 - accuracy: 0.1648
0 2 10000 0.16688918558077437
(5000, 10000) (5000, 1)
50/50 [==============================] - 27s 547ms/step - loss: 20.4788 - accuracy: 0.1816
0 3 15000 0.25033377837116155
(5000, 10000) (5000, 1)
50/50 [==============================] - 29s 572ms/step - loss: 40.8707 - accuracy: 0.0184
0 4 20000 0.33377837116154874
(5000, 10000) (5000, 1)
50/50 [==============================] - 29s 590ms/step - loss: 50.5133 - accuracy: 0.1144
0 5 25000 0.4172229639519359
(5000, 10000) (5000, 1)
50/50 [==============================] - 30s 592ms/step - loss: 3.8044 - accuracy: 0.1434
0 6 30000 0.5006675567423231
(5000, 10000) (5000, 1)
50/50 [==============================] - 34s 674ms/step - loss: 6.9540 - accuracy: 0.2412
0 7 35000 0.58411

In [ ]:
model.save('i10000-8000-o44')